In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
df_team = pd.read_csv("team_stats.csv")
df_player = pd.read_csv("player_stats.csv")

In [3]:
df = pd.merge(df_player,df_team, on=['fixture_id','team_name'],how="inner")

In [4]:
df

,team_name,if_home_team,fixture_id,year_x,avg_minutes_team,avg_rating_team,avg_offsides_team,avg_shots_total_team,avg_shots_on_team,avg_goals_total_team,avg_goals_conceded_team,avg_assists_team,avg_saves_team,avg_passes_total_team,avg_passes_key_team,avg_passes_accuracy_team,avg_tackles_total_team,avg_tackles_blocks_team,avg_tackles_interceptions_team,avg_duels_total_team,avg_duels_won_team,avg_dribbles_attempts_team,avg_dribbles_success_team,avg_dribbles_past_team,avg_fouls_drawn_team,avg_fouls_committed_team,avg_cards_yellow_team,avg_cards_red_team,avg_penalty_won_team,avg_penalty_committed_team,year_y,avg_Gols,avg_Total_Gols,avg_passes_%,avg_shots_on_goal,avg_shots_off_goal,avg_total_shots,avg_blocked_shots,avg_shots_insidebox,avg_shots_outsidebox,avg_fouls,avg_corner_kicks,avg_offsides,avg_ball_possession,avg_goalkeeper_saves,avg_total_passes,avg_passes_accurate
0,Vasco DA Gama,0,35175,2017,45.000000,6.700000,0.000000,2.000000,0.0,0.000000,0.000000,0.000000,0.000000,14.000000,0.000000,93.000000,0.000000,0.000000,0.000000,6.000000,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,2017,1.333333,2.333333,79.000000,3.666667,8.333333,16.000000,4.000000,6.333333,9.666667,15.333333,4.000000,2.000000,44.333333,2.333333,356.000000,282.666667
1,Botafogo,1,35177,2017,79.818182,7.036364,0.000000,9.000000,1.0,0.000000,0.000000,0.000000,2.000000,210.000000,4.000000,803.000000,0.000000,0.000000,12.000000,83.000000,46.000000,7.000000,6.000000,7.000000,17.000000,10.000000,2.000000,0.0,0.000000,0.000000,2017,1.000000,2.000000,75.666667,2.333333,3.000000,7.000000,1.666667,3.333333,3.666667,17.333333,1.333333,0.333333,39.666667,1.666667,338.000000,255.666667
2,Corinthians,0,35177,2017,79.727273,6.754545,0.000000,17.000000,6.0,0.000000,1.000000,0.000000,1.000000,400.000000,10.000000,865.000000,0.000000,0.000000,5.000000,83.000000,42.000000,9.000000,5.000000,5.000000,18.000000,10.000000,1.000000,0.0,0.000000,0.000000,2017,1.333333,2.333333,78.333333,6.000000,4.000000,13.000000,3.000000,8.000000,5.000000,14.666667,5.333333,1.000000,48.666667,0.666667,388.000000,310.666667
3,Chapecoense-sc,1,35178,2017,85.545455,6.990909,0.000000,5.000000,2.0,0.000000,0.000000,0.000000,4.000000,283.000000,3.000000,853.000000,0.000000,0.000000,9.000000,75.000000,37.000000,5.000000,4.000000,2.000000,2.000000,20.000000,4.000000,0.0,0.000000,0.000000,2017,1.000000,2.000000,77.666667,3.000000,2.000000,7.000000,2.000000,2.666667,4.333333,18.333333,2.333333,0.666667,37.333333,3.000000,327.666667,255.666667
4,Fluminense,0,35178,2017,84.375000,6.887500,0.000000,6.000000,3.0,1.000000,1.000000,1.000000,0.000000,346.000000,4.000000,668.000000,0.000000,0.000000,12.000000,80.000000,36.000000,5.000000,1.000000,6.000000,9.000000,14.000000,1.000000,0.0,0.000000,0.000000,2017,1.666667,3.000000,85.333333,3.000000,7.333333,13.666667,3.333333,6.000000,8.000000,15.333333,4.666667,1.333333,62.333333,2.333333,502.333333,429.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5710,Internacional,0,1180732,2024,66.878788,6.830303,1.000000,7.666667,4.0,0.666667,1.666667,1.666667,2.333333,306.666667,5.000000,252.333333,16.333333,0.666667,7.333333,74.000000,35.000000,11.000000,4.333333,5.666667,8.000000,10.666667,2.000000,0.0,0.333333,0.000000,2024,2.000000,3.666667,82.666667,6.333333,4.666667,14.333333,3.333333,9.333333,5.000000,12.333333,7.333333,1.000000,55.333333,2.333333,422.000000,350.000000
5711,Cuiaba,1,1180733,2024,66.619048,6.595238,0.333333,4.000000,2.0,1.000000,0.000000,0.000000,0.000000,147.000000,3.000000,123.000000,11.000000,2.333333,3.666667,48.666667,24.666667,7.333333,3.000000,4.000000,6.333333,4.000000,0.666667,0.0,0.000000,0.000000,2024,0.666667,2.000000,81.000000,2.000000,3.666667,7.000000,1.333333,3.666667,3.333333,8.333333,3.666667,2.333333,42.000000,3.000000,370.666667,301.666667
5712,Vasco

In [5]:
# Passo 1: Pivotar o DataFrame
df_pivot = df.set_index(['fixture_id', 'if_home_team']).unstack('if_home_team')

# Passo 2: Renomear colunas
df_pivot.columns = [
    f"{col}_{'home' if home else 'away'}" 
    for col, home in df_pivot.columns
]
df_pivot

,team_name_away,team_name_home,year_x_away,year_x_home,avg_minutes_team_away,avg_minutes_team_home,avg_rating_team_away,avg_rating_team_home,avg_offsides_team_away,avg_offsides_team_home,avg_shots_total_team_away,avg_shots_total_team_home,avg_shots_on_team_away,avg_shots_on_team_home,avg_goals_total_team_away,avg_goals_total_team_home,avg_goals_conceded_team_away,avg_goals_conceded_team_home,avg_assists_team_away,avg_assists_team_home,avg_saves_team_away,avg_saves_team_home,avg_passes_total_team_away,avg_passes_total_team_home,avg_passes_key_team_away,avg_passes_key_team_home,avg_passes_accuracy_team_away,avg_passes_accuracy_team_home,avg_tackles_total_team_away,avg_tackles_total_team_home,avg_tackles_blocks_team_away,avg_tackles_blocks_team_home,avg_tackles_interceptions_team_away,avg_tackles_interceptions_team_home,avg_duels_total_team_away,avg_duels_total_team_home,avg_duels_won_team_away,avg_duels_won_team_home,avg_dribbles_attempts_team_away,avg_dribbles_attempts_team_home,avg_dribbles_success_team_away,avg_dribbles_success_team_home,avg_dribbles_past_team_away,avg_dribbles_past_team_home,avg_fouls_drawn_team_away,avg_fouls_drawn_team_home,avg_fouls_committed_team_away,avg_fouls_committed_team_home,avg_cards_yellow_team_away,avg_cards_yellow_team_home,avg_cards_red_team_away,avg_cards_red_team_home,avg_penalty_won_team_away,avg_penalty_won_team_home,avg_penalty_committed_team_away,avg_penalty_committed_team_home,year_y_away,year_y_home,avg_Gols_away,avg_Gols_home,avg_Total_Gols_away,avg_Total_Gols_home,avg_passes_%_away,avg_passes_%_home,avg_shots_on_goal_away,avg_shots_on_goal_home,avg_shots_off_goal_away,avg_shots_off_goal_home,avg_total_shots_away,avg_total_shots_home,avg_blocked_shots_away,avg_blocked_shots_home,avg_shots_insidebox_away,avg_shots_insidebox_home,avg_shots_outsidebox_away,avg_shots_outsidebox_home,avg_fouls_away,avg_fouls_home,avg_corner_kicks_away,avg_corner_kicks_home,avg_offsides_away,avg_offsides_home,avg_ball_possession_away,avg_ball_possession_home,avg_goalkeeper_saves_away,avg_goalkeeper_saves_home,avg_total_passes_away,avg_total_passes_home,avg_passes_accurate_away,avg_passes_accurate_home
fixture_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
35175,Vasco DA Gama,NaN,2017.0,NaN,45.000000,NaN,6.700000,NaN,0.000000,NaN,2.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,14.000000,NaN,0.000000,NaN,93.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,6.000000,NaN,2.000000,NaN,1.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,1.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,2017.0,NaN,1.333333,NaN,2.333333,NaN,79.000000,NaN,3.666667,NaN,8.333333,NaN,16.000000,NaN,4.000000,NaN,6.333333,NaN,9.666667,NaN,15.333333,NaN,4.000000,NaN,2.000000,NaN,44.333333,NaN,2.333333,NaN,356.000000,NaN,282.666667,NaN
35177,Corinthians,Botafogo,2017.0,2017.0,79.727273,79.818182,6.754545,7.036364,0.000000,0.000000,17.000000,9.000000,6.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,400.000000,210.000000,10.000000,4.000000,865.000000,803.000000,0.000000,0.000000,0.000000,0.000000,5.000000,12.000000,83.000000,83.000000,42.000000,46.000000,9.000000,7.000000,5.000000,6.000000,5.000000,7.000000,18.000000,17.000000,10.000000,10.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2017.0,2017.0,1.333333,1.000000,2.333333,2.000000,78.333333,75.666667,6.000000,2.333333,4.000000,3.000000,13.000000,7.000000,3.000000,1.666667,8.000000,3.333333,5.000000,3.666667,14.666667,17.333333,5.333333,1.333333,1.000000,0.333333,48.666667,39.666667,0.666667,1.666667,388.000000,338.000000,310.666667,255.666667
35178,Fluminense,Chapecoense-sc,2017.0,2017.0,84.375000,85.545455,6.887500,6.990909,0.000000,0.000000,6.000000,5.000000,3.0,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,4.000000,346.000000,283.000000,4.000000,3.000000,668.000000,853.000000,0.000000,0.000000,0.000000,0.000000,12

# Construção da variável respota

In [6]:
# Lista para armazenar os DataFrames de cada ano
dfs = []

# Loop pelos anos de 2017 a 2024
for year in range(2017, 2025):
    # Monta o caminho do arquivo para cada ano
    file_path = f"../Data_Lake/Camp_Brasileiro/{year}/Camp_Brasileiro_{year}_Games_final.csv"

    # Carrega o DataFrame do ano atual e adiciona à lista
    df_temp = pd.read_csv(file_path)
    dfs.append(df_temp)

# Concatena todos os DataFrames em um único
df_Games = pd.concat(dfs, ignore_index=True)

In [7]:
df_Games['Total_Gols'] = df_Games['Gols_Home'] + df_Games['Gols_Away']
df_tot_gols = df_Games[['fixture_id','Total_Gols']]
df_pivot = pd.merge(df_pivot,df_tot_gols, on='fixture_id')
df_pivot['Total_Gols'] = df_pivot['Total_Gols'] > 2.5    #Mais de 2.5 gols = true = 1
df_pivot['Total_Gols'] = df_pivot['Total_Gols'].astype(int)

In [8]:
df = df_pivot

In [9]:
df.dropna(inplace=True)
df = df.drop(['year_y_away', 'year_y_home'], axis=1)

In [11]:
df.columns[5:-2].tolist()

['avg_minutes_team_away',
 'avg_minutes_team_home',
 'avg_rating_team_away',
 'avg_rating_team_home',
 'avg_offsides_team_away',
 'avg_offsides_team_home',
 'avg_shots_total_team_away',
 'avg_shots_total_team_home',
 'avg_shots_on_team_away',
 'avg_shots_on_team_home',
 'avg_goals_total_team_away',
 'avg_goals_total_team_home',
 'avg_goals_conceded_team_away',
 'avg_goals_conceded_team_home',
 'avg_assists_team_away',
 'avg_assists_team_home',
 'avg_saves_team_away',
 'avg_saves_team_home',
 'avg_passes_total_team_away',
 'avg_passes_total_team_home',
 'avg_passes_key_team_away',
 'avg_passes_key_team_home',
 'avg_passes_accuracy_team_away',
 'avg_passes_accuracy_team_home',
 'avg_tackles_total_team_away',
 'avg_tackles_total_team_home',
 'avg_tackles_blocks_team_away',
 'avg_tackles_blocks_team_home',
 'avg_tackles_interceptions_team_away',
 'avg_tackles_interceptions_team_home',
 'avg_duels_total_team_away',
 'avg_duels_total_team_home',
 'avg_duels_won_team_away',
 'avg_duels_won_te

# Modelo

In [50]:
# 1. Definindo as features e o target
features = df.columns[3:-2].tolist()  # Seleciona as colunas de features
target = 'Total_Gols'  # Define a coluna target

X = df[features]  # Features
y = df[target]    # Target

# 2. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Definir o modelo de Random Forest para classificação
model = RandomForestClassifier(random_state=42)

# 4. Definir os parâmetros para o GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árvores na floresta
    'max_depth': [None, 10, 20, 30],  # Profundidade máxima das árvores
    'min_samples_split': [2, 5, 10],  # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],    # Número mínimo de amostras em uma folha
    'max_features': ['auto', 'sqrt']  # Número de features consideradas para divisão
}

# 5. Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',  # Métrica de avaliação (acurácia)
    cv=3,  # Número de folds na validação cruzada
    n_jobs=-1,  # Usar todos os núcleos do processador
    verbose=2  # Mostrar logs durante o treinamento
)

# 6. Treinar o modelo com GridSearchCV
grid_search.fit(X_train, y_train)

# 7. Melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)

# 8. Melhor modelo
best_model = grid_search.best_estimator_

# 9. Fazer previsões com o melhor modelo
y_pred = best_model.predict(X_test)

#---
y_train_pred = best_model.predict(X_train)
y_train_prob = best_model.predict_proba(X_train)
y_test_pred = best_model.predict(X_test)
y_test_prob = best_model.predict_proba(X_test)
print(f'Acurácia treino: {metrics.accuracy_score(y_train, y_train_pred):.4f}')
print(f'Acurácia teste: {metrics.accuracy_score(y_test, y_test_pred):.4f}')
print(f'AUC treino: {metrics.roc_auc_score(y_train, y_train_prob[:,1]):.4f}')
print(f'AUC teste: {metrics.roc_auc_score(y_test, y_test_prob[:,1]):.4f}')


Fitting 3 folds for each of 216 candidates, totalling 648 fits
Melhores parâmetros encontrados:
{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Acurácia treino: 1.0000
Acurácia teste: 0.5541
AUC treino: 1.0000
AUC teste: 0.5563


c:\Users\paulo\OneDrive\Área de Trabalho\projetos\Footballab\.venv\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
324 fits failed out of a total of 648.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
163 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\paulo\OneDrive\Área de Trabalho\projetos\Footballab\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\paulo\OneDrive\Área de Trabalho\projetos\Footballab\.venv\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\paulo\OneDrive\Área de Trabalho\projetos